# Script to Create Figure 2 - Maps of Example Nearshore Area for Benguela Subregions

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib import colors
from matplotlib import cm
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import datetime as dt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import math
import warnings 
warnings.simplefilter('ignore') 
import seaborn as sns
import glob

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from calendar import month_abbr
from geopy import distance

from matplotlib.ticker import StrMethodFormatter
from matplotlib import colors as c
#from matplotlib_scalebar.scalebar import ScaleBar
from datetime import datetime
import matplotlib.dates as mdates
from scalebar import scale_bar
from datetime import datetime

#specify data from Google Drive
ddir = '../../data/data_download'

In [ ]:
###Benguela map with 300 and 30 km zone
#read in new climatology file
cali = xr.open_dataset(ddir+'/California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()

beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()

beng_coast = xr.open_dataset(ddir + 'Offshore_Onshorebands/Benguela_NearshoreMonthlyMean_30km.nc')
beng_coast.close()

iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,30],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-9],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 19.5],[10, 17],[8, 15]])

#select out sst cutoff
sst_upw = [15.6278657913208, 15.074623107910156, 18.181407928466797] #nearshore means (modified subregions)

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(beng.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(beng.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(beng.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(beng.analysed_sst.lon)])

latr = [-37, -27]
lonr = [12, 20]

#create a region variable with a margin
margin = 1
#region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 300) 
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines(resolution='10m',linewidth=2,color='black', zorder = 1) 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'dimgrey', linewidth = 2, zorder = 0)
ax.set_xticks(np.round([*np.arange(region_raw[1,0],region_raw[1,1]+1,2)],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0])+1,region[0,1]+1,2)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
#ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
ax.tick_params('both', labelsize=14, width = 2, length = 6)

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

#plot the 300km zone
cmap = ListedColormap('royalblue') #matplotlib.colors.LinearSegmentedColormap.from_list("", ["blue"])
fg = beng.sel(lat = slice(beng_lats[0][0], beng_lats[0][1])).analysed_sst.plot(cmap = cmap, add_colorbar=False, zorder = 1)

#plot the 30km zone
cmap = ListedColormap('firebrick')
fg = beng_coast.isel(time = 0).sel(lat = slice(beng_lats[0][0], beng_lats[0][1])).analysed_sst.plot(x = 'lon', y = 'lat', cmap = cmap, add_colorbar=False, zorder = 2)
plt.title('')

#add colorbar
# cb = plt.colorbar(fg, pad = 0.02)      
# cb.set_label(label = 'Sea Surface Temperature (SST, °C)', size = 16, labelpad = 10)
# cb.ax.tick_params(labelsize = 14)

rect2 = patches.Rectangle((beng_lons[0][0], beng_lats[0][0]), (beng_lons[0][1]-beng_lons[0][0]), (beng_lats[0][1]-beng_lats[0][0]), linewidth=2, edgecolor='k', facecolor='none', zorder = 3)
ax.add_patch(rect2)

#add in the green line showing the area 
# try:
#     beng.sel(lat = slice(beng_lats[i][0], beng_lats[i][1]), lon = slice(beng_lons[i][0], beng_lons[i][1])).analysed_sst.plot.contour(add_colorbar = False, levels=[sst_upw[i]], colors=['limegreen'], linewidths = [5])
# except:
#     pass


#add scalebar
scale_bar(ax, (0.1, 0.04), 2_50, text_kwargs = {'fontsize': 14})

scale_bar(ax, (0.5, 0.188), 3_00, color = 'None', text_offset = -0.045, linewidth = 4, text_kwargs = {'fontsize': 16, 'color':'royalblue'})
scale_bar(ax, (0.548, 0.188), 2_10, color = 'royalblue', text_offset = -0.045, linewidth = 4, text_kwargs = {'fontsize': 16, 'color':'None'})

plt.arrow(18.35, -35.735, 1, 0, color = 'royalblue', shape='full', lw=0, length_includes_head=True, head_width=.4, zorder = 6)
plt.arrow(16.85, -35.735, -1, 0, color = 'royalblue', shape='full', lw=0, length_includes_head=True, head_width=.4, zorder = 6)

#add north arrow
x, y, arrow_length = 0.119, 0.175, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(color='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)

#add 30km annotation
ax.annotate('30-km band', xy=(0.53, 0.76), xytext=(0.3, 0.85),
            arrowprops=dict(facecolor='firebrick', edgecolor = 'black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16, color = 'firebrick',
            xycoords=ax.transAxes)

plt.xlabel('Longitude', fontsize = 18)
plt.ylabel('Latitude', fontsize = 18)
plt.title('(A) Full and Nearshore Bands', fontsize = 24, loc = 'left')
[x.set_linewidth(2) for x in ax.spines.values()]
plt.tight_layout()
plt.savefig('../../figures/Benguela_map_bands.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()


In [ ]:
###Benguela map with SST and threshold
#read in new climatology file
cali = xr.open_dataset(ddir+'/California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()

beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()

beng_mon = xr.open_dataset(ddir + 'Benguela/beng_MURR_2016.nc')
beng_mon.close()

iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,30],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-9],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 19.5],[10, 17],[8, 15]])

#select out sst cutoff
sst_upw = [15.6278657913208, 15.074623107910156, 18.181407928466797] #nearshore means (modified subregions)

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(beng.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(beng.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(beng.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(beng.analysed_sst.lon)])

latr = [-37, -27]
lonr = [12, 20]

#create a region variable with a margin
margin = 1
#region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 300) 
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines(resolution='10m',linewidth=2,color='black', zorder = 1) 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'dimgrey', linewidth = 2, zorder = 0)
ax.set_xticks(np.round([*np.arange(region_raw[1,0],region_raw[1,1]+1,2)],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0])+1,region[0,1]+1,2)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
#ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
ax.tick_params('both', labelsize=14, width = 2, length = 6)

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

#create custom colormap 
seismic = cm.get_cmap('seismic', 256)
newcolors = seismic(np.linspace(0, 1, 256))
limegreen = np.array([50/256, 205/256, 50/256, 1])
newcolors[127:129, :] = limegreen
newcmp = ListedColormap(newcolors)

#normalize based on min, max, and center value
divnorm=colors.TwoSlopeNorm(vmin=11, vcenter=sst_upw[0], vmax=21)

#plot the monthly data
fg = beng_mon.where(beng_mon.time.dt.month == 9, drop = True).sel(lat = slice(beng_lats[0][0], beng_lats[0][1])).analysed_sst.plot(cmap = newcmp, norm = divnorm, add_colorbar=False, zorder = 1, vmin = 11, vmax = 21)
beng_mon.where(beng_mon.time.dt.month == 9, drop = True).sel(lat = slice(beng_lats[0][0], beng_lats[0][1])).analysed_sst.plot(cmap = 'seismic', norm = divnorm, add_colorbar=False, zorder = 1, vmin = 11, vmax = 21)
plt.title('')
ax.annotate('SST threshold: \n' + str(np.around(sst_upw[0], 1)) + ' °C', xy = (0.02, 0.95), 
            ha = 'left', va = 'center', fontsize = 18, xycoords = ax.transAxes)

#add colorbar
cb = plt.colorbar(fg, pad = 0.02)      
cb.set_label(label = 'Sea Surface Temperature (SST, °C)', size = 16, labelpad = 10)
cb.ax.tick_params(labelsize = 14)

rect2 = patches.Rectangle((beng_lons[0][0], beng_lats[0][0]), (beng_lons[0][1]-beng_lons[0][0]), (beng_lats[0][1]-beng_lats[0][0]), linewidth=2, edgecolor='k', facecolor='none', zorder = 3)
ax.add_patch(rect2)

beng_mon.where(beng_mon.time.dt.month == 9, drop = True).sel(lat = slice(beng_lats[0][0], beng_lats[0][1]), lon = slice(beng_lons[0][0], beng_lons[0][1])).analysed_sst.to_dataframe().reset_index('time').drop('time', axis = 1).to_xarray().analysed_sst.plot.contour(add_colorbar = False, levels=[sst_upw[0]], colors=['limegreen'], linewidths = [2])

#add scalebar
scale_bar(ax, (0.1, 0.04), 2_50, text_kwargs = {'fontsize': 14})

#add north arrow
x, y, arrow_length = 0.119, 0.175, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(color='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)


plt.xlabel('Longitude', fontsize = 18)
plt.ylabel('Latitude', fontsize = 18)
plt.title('(B) MUR SST September 2016', fontsize = 24, loc = 'left')
[x.set_linewidth(2) for x in ax.spines.values()]
plt.tight_layout()
plt.savefig('../../figures/Benguela_map_sstthreshold.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()
